## Индексы

Индексы нужны для повышения производительности поиска данных. Ускорение работы достигается в основном за счёт того, что записи в индексе упорядочены и имеют структуру, оптимизированную под поиск — например, сбалансированного дерева.

При создании индекса значения атрибутов, участвующих в нем, могут комбинироваться и преобразовываться различными способами.

### Типы индексов

Существует два типа индексов БД по способу хранения сортировки данных:

1. Кластерные индексы. При наличии кластерного индекса строки таблицы упорядочены по значению ключа этого индекса. Если в таблице нет кластерного индекса, таблица называется кучей.

2. Некластерные индексы. Содержит только указатели на записи таблицы. 

Кластерный индекс может быть только одним для каждой таблицы, но каждая таблица может иметь несколько различных некластерных индексов, каждый из которых определяет свой собственный порядок следования записей.

Индексы могут быть реализованы различными структурами. Наиболее часто употребимы B*-деревья, B+-деревья, B-деревья и хеши.


### Использование индекса при поиске строк

Планировщик проверяет, имеется ли для этой таблицы индекс, созданный на основе тех же столбцов, что указаны, например, в условии предложения **WHERE**. 
- Если индекс существует, то планировщик оценивает целесообразность его использования в данном конкретном случае.
- Если его использование целесообразно, то сначала выполняется поиск необходимых значений в индексе, а затем, если такие значения в нем найдены, производится обращение к таблице с использованием указателей, которые хранятся в записях индекса.
- Таким образом, полный перебор строк в таблице может быть заменен поиском в упорядоченном индексе и переходом к строке таблицы по прямому указателю (ссылке).

Примечание: 

Индексы требуют некоторых накладных расходов на их создание и поддержание в актуальном состоянии при выполнении обновлений данных в таблицах.

Индекс, созданный по столбцу, участвующему в соединении двух таблиц, может позволить ускорить процесс выборки записей из таблиц.

#### Пример

Включим в утилите psql секундомер: \timing on

    SELECT count( product_id ) FROM products
    WHERE brand = 'БольшойЖивот';

    CREATE INDEX product ON products (product_id, brand);
    
Автоматически создаются по умолчанию тип индекса B-деревоиндексы, ограничения PRIMARY KEY и UNIQUE.

    SELECT count( product_id ) FROM products
    WHERE brand = 'БольшойЖивот'; 
    
Посмотреть список всех индексов в текущей базе данных: \di или \di+
    
Для удаления индекса используется команда: 
    
    DROP INDEX product
    
### Ключ

Первичный ключ -это набор определенных признаков, уникальных для каждой записи. Обозначается первичный ключ, как primary key.

Primary key важен потому что
- Primary key не позволяет создавать одинаковых записей (строк) в таблице;
- PK обеспечивают логическую связь между таблицами одной базы данных (для реляционных БД).

Первичный ключ может быть:

- естественным – существует в реальном мире, например ФИО, или номер и серия паспорта;
- суррогатным – не существует в реальном мире, например какой-то порядковый номер, который существует только в базе данных;
- составным.

Внешний ключ - это логическую связь, между таблицами одной БД. Foreign key не может ссылаться только на определённые поля другой таблицы.

### Создание уникальных индексов

Индексы могут использоваться для обеспечения уникальности значений атрибутов в строках таблицы. 

    CREATE UNIQUE INDEX customers_unique_name_key ON customers ( name );

Примечание: В уникальных индексах допускается наличие значений NULL, поскольку они считаются не совпадающими ни с какими другими 
значениями, в том числе и друг с другом.

Если уникальный индекс создан по нескольким атрибутам, то совпадающими считаются лишь те комбинации значений атрибутов в двух строках, в которых совпадают значения всех соответственных атрибутов.

### Использование функции при создании индекса

В команде создания индекса можно использовать не только имена столбцов, но также функции и скалярные выражения, построенные на основе столбцов таблицы.

#### Пример

Запретим значения столбца name в таблице customers, отличающиеся только регистром символов.

    CREATE UNIQUE INDEX customers_unique_name_key ON customers ( lower( name ) );
    
Индекс строится уже на основе преобразованных значений, поэтому при поиске строки в таблице искомое значение сначала переводится в нижний регистр, а затем осуществляется поиск в индексе.
То есть для имени Макс существует ключ "(lower(name))=(макс)".

Примечание: 

- Индексы на основе выражений требуют больше ресурсов для их создания и поддержания при вставке и обновлении строк в таблице.
- При выполнении выборок, построенных на основе сложных выражений, работа происходит с меньшими накладными расходами, поскольку в индексе хранятся уже вычисленные значения этих выражений, пусть даже самых сложных.
- Поэтому такие индексы целесообразно использовать тогда, когда выборки производятся многократно, и время, затраченное на создание и поддержание индекса, компенсируется (окупается) при выполнении выборок из таблицы.

## Кэш запросов

Кэш запросов - это функция, которая позволяет ускорить извлечение данных из БД. 

План выполнения запроса помещается в кэш и ассоциируется с синтаксическим деревом или текстом запроса. Впоследствии, если семантика входящего запроса соответствует семантике некоторого запроса, помещённого в кэш, то СУБД использует сохранённый план выполнения, а не генерирует его.

Перед помещением в кэш запрос очищается от элементов, не влияющих на структуру плана. Такими элементами могут быть, например, константы.

По сравнению с данными, хранящимися на диске, кэшированные данные из RAM (Random Access Memory) можно получить быстрее, что улучшает операции ввода/вывода.

## Где потренироваться?

www.sql-tutorial.ru + www.sql-ex.ru - Теория и практика

http://sqlfiddle.com/ – Можно выбрать БД, создать таблицы, заполнить, кликнуть Build Schema. Затем пробовать разные запросы.


# Источник

http://www.sql-tutorial.ru/ru/book_sql_dml.html
https://edu.postgrespro.ru/sqlprimer/sqlprimer-2019-msu-06.pdf
https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D0%B4%D0%B5%D0%BA%D1%81_%28%D0%B1%D0%B0%D0%B7%D1%8B_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%29